In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import cupy as cp

In [ ]:
# Preparar los datos (secuencial)
# código de: https://keras.io/examples/vision/mnist_convnet/

# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range in GPU
#x_train = x_train.astype("float32") / 255
#x_test = x_test.astype("float32") / 255

x_train_gpu=cp.asarray(x_train)
x_train_gpu=cp.divide(x_train_gpu, 255.0)
x_train = cp.asnumpy(x_train_gpu)

x_test_gpu=cp.asarray(x_test)
x_test_gpu=cp.divide(x_test_gpu, 255.0)
x_test = cp.asnumpy(x_test_gpu)

# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

11490434/11490434 [==============================] - 0s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [ ]:
# Se crea el modelo
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dropout (Dropout)           (None, 1600)              0

In [ ]:
# Se entrena el modelo
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/15
422/422 [==============================] - 11s 6ms/step - loss: 0.3619 - accuracy: 0.8903 - val_loss: 0.0819 - val_accuracy: 0.9772
Epoch 2/15
422/422 [==============================] - 2s 5ms/step - loss: 0.1117 - accuracy: 0.9658 - val_loss: 0.0593 - val_accuracy: 0.9822
Epoch 3/15
422/422 [==============================] - 2s 5ms/step - loss: 0.0857 - accuracy: 0.9738 - val_loss: 0.0475 - val_accuracy: 0.9868
Epoch 4/15
422/422 [==============================] - 2s 5ms/step - loss: 0.0711 - accuracy: 0.9779 - val_loss: 0.0437 - val_accuracy: 0.9875
Epoch 5/15
422/422 [==============================] - 2s 5ms/step - loss: 0.0622 - accuracy: 0.9803 - val_loss: 0.0412 - val_accuracy: 0.9888
Epoch 6/15
422/422 [==============================] - 2s 5ms/step - loss: 0.0554 - accuracy: 0.9826 - val_loss: 0.0362 - val_accuracy: 0.9900
Epoch 7/15
422/422 [==============================] - 2s 5ms/step - loss: 0.0528 - accuracy: 0.9836 - val_loss: 0.0357 - val_accuracy: 0.9898
Epoch

In [ ]:
# Evaluación del modelo
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])


Test loss: 0.024299558252096176
Test accuracy: 0.9919000267982483
